In [2]:
from dotenv import load_dotenv, dotenv_values
import os

# Load environment variables from .env file
config = dotenv_values("C:/Users/SACHENDRA/Documents/Activeloop/.env")
load_dotenv("C:/Users/SACHENDRA/Documents/Activeloop/.env")

True

In [3]:
import os
import getpass

from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

# os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')
# os.environ['ACTIVELOOP_TOKEN'] = getpass.getpass('Activeloop Token:')
embeddings = OpenAIEmbeddings()

In [4]:
db = DeepLake(dataset_path="hub://davitbun/twitter-algorithm", read_only=True, embedding_function=embeddings)


Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Deep Lake Dataset in hub://davitbun/twitter-algorithm already exists, loading from the storage


In [5]:
retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['fetch_k'] = 100
retriever.search_kwargs['k'] = 10

In [6]:
def filter(x):
    if 'com.google' in x['text'].data()['value']:
        return False
    metadata = x['metadata'].data()['value']
    return 'scala' in metadata['source'] or 'py' in metadata['source']

# Uncomment the following line to apply custom filtering
# retriever.search_kwargs['filter'] = filter

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model='gpt-3.5-turbo') # switch to 'gpt-4'
qa = ConversationalRetrievalChain.from_llm(model,retriever=retriever)

C:\Users\SACHENDRA\miniconda3\envs\activeloop2\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [8]:
questions = [
    "What does favCountParams do?",
    "is it Likes + Bookmarks, or not clear from the code?",
    "What are the major negative modifiers that lower your linear ranking parameters?",   
    "How do you get assigned to SimClusters?",
    "What is needed to migrate from one SimClusters to another SimClusters?",
    "How much do I get boosted within my cluster?",   
    "How does Heavy ranker work. what are it’s main inputs?",
    "How can one influence Heavy ranker?",
    "why threads and long tweets do so well on the platform?",
    "Are thread and long tweet creators building a following that reacts to only threads?",
    "Do you need to follow different strategies to get most followers vs to get most likes and bookmarks per tweet?",
    "Content meta data and how it impacts virality (e.g. ALT in images).",
    "What are some unexpected fingerprints for spam factors?",
    "Is there any difference between company verified checkmarks and blue verified individual checkmarks?",
] 
chat_history = []

for question in questions:  
    result = qa.invoke({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")



-> **Question**: What does favCountParams do? 

**Answer**: The `favCountParams` is a parameter of type `ThriftLinearFeatureRankingParams` that likely relates to favorited counts of tweets in a ranking or scoring system. It is used to rank or score tweets based on the number of favorites they have received. 

-> **Question**: is it Likes + Bookmarks, or not clear from the code? 

**Answer**: Based on the provided code snippets and context, it seems that `favCountParams` specifically refers to the count of favorites (likes) on tweets. There is no mention of bookmarks in the context provided. 

-> **Question**: What are the major negative modifiers that lower your linear ranking parameters? 

**Answer**: The major negative modifiers that lower the linear ranking parameters are:

1. Social Filter
2. Direct Follow Boost
3. Trusted Circle Boost
4. Out of Network Reply Penalty
5. No Text Hit Demotion
6. URL Only Hit Demotion
7. Name Only Hit Demotion
8. Separate Text and Name Hit Demotion
9.